In [1]:
from bs4 import BeautifulSoup
import csv
import re

In [2]:
file_path = "data/Resume.csv"
rows = []
with open(file_path, 'r', encoding='utf-8') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
    
    for row in csvreader:
        rows.append(row)

In [3]:
def data_clean(text):
    text = text.lower()

    # remove non-alphabetical characters
    REGEX = re.compile("[^ ^a-zA-Z]")
    text = re.sub(REGEX, ' ', text)
    text = re.sub('\n', ' ', text)

    # remove extra spaces
    wds = text.split(' ')
    text = ' '.join(wds)
    
    text = text.strip()
    return text

In [4]:
def is_expr(sec, title):
    if sec == "EXPR":
        return True
    if "work" in title or "experience" in title:
        return True
    
    return False

In [5]:
def extract_content(sec):
    ct = sec.find_all("div", {"class": "paragraph"})
    rt = []
    for p in ct:
        for s in p.stripped_strings:
            s = s.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
            rt.append(s)
    return rt

In [6]:
def extract_expr(sec):
    experiences = sec.find_all("div", {"class":"singlecolumn"})
    exprs = []
    for e in experiences:
        dt = {}

        jobtitle = e.find("span", {"class":"jobtitle"})
        if jobtitle:
            jobtitle = jobtitle.text.strip().replace('\n', ' ')
        dt['jobtitle'] = jobtitle

        dates = e.find_all("span", {"class":"jobdates"})
        startsdate = ""
        endsdate = ""
        if dates:
            startsdate = dates[0].text.strip()
            if len(dates) == 2:
                endsdate = dates[1].text.strip()
        dt['startsdate'] = startsdate
        dt['endsdate'] = endsdate

        joblines = e.find("span", {"class":"jobline"})
        jobduty = []
        if joblines:
            for s in joblines.stripped_strings:
                s = s.replace('\n', ' ').replace('\r', ' ').replace('\t', '')
                jobduty.append(s)
        dt['jobduty'] = jobduty

        exprs.append(dt)
    return exprs



In [7]:
from collections import defaultdict
import json
freq = defaultdict(int)
cb = defaultdict(int)
data = {}

idx = 0
section = defaultdict(set)
experiences = []
for row in rows:
    if not row[1]:
        break
    data["row"+str(idx)] = []
    soup = BeautifulSoup(row[2], 'html.parser')
    comb = []
    a = soup.find_all("div", {"class": "section"})
    exprs = {}
    exprs["id"] = row[0]
    exprs["experiences"] = []
    for s in a:
        b = s.find("div", {"class": "sectiontitle"})
        
        if b:
            title = b.text.strip().lower()
            if len(title) > 0: 
                section[b['id'][9:13]].add(title)
            if is_expr(b['id'], title):
                exprs["experiences"] = extract_expr(s)

            dt = {}
            freq[title] += 1
            comb.append(title)
            dt["id"] = row[0]
            dt["title"] = title
            dt["content"] = extract_content(s)
            dt["category"] = row[3]
            if dt:
                data["row"+str(idx)].append(dt)
    comb.sort()
    k = "_".join(comb)
    cb[k] += 1
    idx += 1 
    experiences.append(json.dumps(exprs))


In [8]:
section

defaultdict(set,
            {'SUMM': {'about',
              'background, objective',
              'career focus',
              'career objective',
              'career overview',
              'engineering director',
              'executive profile',
              'executive summary',
              'floral designerwrite your own title',
              'focus - graphic design / digital art',
              'objective',
              'objective statement',
              'overview',
              'personal summary',
              'position applying',
              'professional background',
              'professional development',
              'professional objective',
              'professional overview',
              'professional profile',
              'professional summary',
              'profile',
              'risk mangement leader',
              'sales & account professional',
              'senior accountant',
              'summary',
              'summary of qualific

In [9]:
expr_file = "output/experiences.txt"
with open(expr_file, "w", encoding='utf-8') as f:
    for ex in experiences:
        f.write(f"{ex}\n")


In [10]:
print(data["row2000"])

[{'id': '12212468', 'title': 'summary', 'content': ["Highly motivated Civil Engineer with 16 years of experience working for one of North America's largest and most respected construction and engineering organizations with revenues of more than $11 billion and consistently ranking among the top five contractors by Engineering News-Record.", 'My experience with Kiewit has prepared me to successfully plan and execute projects while promoting safety, quality, schedule, and cost performance. My career experience ranges from construction design coordinator on a multi billion dollar design build project, to on-site project engineering and project management for self-performing heavy civil construction work ranging from $545 million to $3.1 million dollars. Excellent communicator skilled in construction means and methods, cost controls, scheduling, claims analysis and contract administration.', 'As the Field Engineer and Project Controls Manager on one of the largest design-build project in M

In [11]:
s = BeautifulSoup(rows[2000][2], 'html.parser')
print(s.prettify())

<div class="fontsize fontface vmargins hmargins linespacing pagesize" id="document">
 <div class="section firstsection" id="SECTION_NAME585766938" style="
      padding-top:0px;
    ">
  <div class="paragraph PARAGRAPH_NAME firstparagraph" id="PARAGRAPH_585766938_1_381272156" style="
      padding-top:0px;
    ">
   <div class="name thinbottomborder" itemprop="name">
    <span class="field fName" id="585766938FNAM1">
    </span>
    <span>
    </span>
    <span class="field" id="585766938LNAM1">
     ASSISTANT PROJECT MANAGER/DESIGN BUILD CONSTRUCTION COORDINATOR
    </span>
   </div>
  </div>
 </div>
 <div class="section" id="SECTION_SUMM585766940" style="
      padding-top:0px;
    ">
  <div class="heading">
   <div class="sectiontitle" id="SECTNAME_SUMM585766940">
    Summary
   </div>
  </div>
  <div class="paragraph firstparagraph" id="PARAGRAPH_585766940_1_381270288" style="
      padding-top:0px;
    ">
   <div class="field singlecolumn noPind" id="585766940FRFM1">
    <p align=

In [12]:
print(s.get_text())

         ASSISTANT PROJECT MANAGER/DESIGN BUILD CONSTRUCTION COORDINATOR       Summary     Highly motivated Civil Engineer with 16 years of experience working for one of North America's largest and most respected construction and engineering organizations with revenues of more than $11 billion and consistently ranking among the top five contractors by Engineering News-Record.   My experience with Kiewit has prepared me to successfully plan and execute projects while promoting safety, quality, schedule, and cost performance. My career experience ranges from construction design coordinator on a multi billion dollar design build project, to on-site project engineering and project management for self-performing heavy civil construction work ranging from $545 million to $3.1 million dollars. Excellent communicator skilled in construction means and methods, cost controls, scheduling, claims analysis and contract administration.   As the Field Engineer and Project Controls Manager on one of t

In [13]:
fq = list(freq.items())
fq.sort(key= lambda x: x[1], reverse=True)
print(fq)

[('skills', 2700), ('education', 1908), ('experience', 1714), ('summary', 1344), ('highlights', 876), ('accomplishments', 761), ('education and training', 508), ('additional information', 462), ('professional summary', 460), ('professional experience', 402), ('work history', 351), ('certifications', 283), ('languages', 238), ('interests', 234), ('professional affiliations', 211), ('core qualifications', 208), ('skill highlights', 187), ('affiliations', 156), ('executive profile', 131), ('personal information', 130), ('work experience', 128), ('qualifications', 100), ('core accomplishments', 93), ('career overview', 85), ('activities and honors', 58), ('professional profile', 51), ('career focus', 48), ('profile', 47), ('technical skills', 44), ('publications', 43), ('presentations', 39), ('core strengths', 35), ('executive summary', 34), ('summary of skills', 34), ('educational background', 33), ('relevant experience', 29), ('military experience', 27), ('objective', 27), ('computer ski

In [14]:
c = list(cb.items())
c.sort(key= lambda x: x[1], reverse=True)
print(c)

[('accomplishments_education_experience_highlights_skills_summary', 195), ('education_experience_highlights_skills_summary', 175), ('education_experience_skills_summary', 74), ('education and training_experience_skills_summary', 55), ('education and training_experience_skills_skills_summary', 53), ('education_professional summary_skills_skills_work history', 51), ('core qualifications_education_experience_professional summary_skills', 34), ('core accomplishments_education_executive profile_professional experience_skill highlights_skills', 29), ('education_experience_highlights_languages_skills_summary', 27), ('accomplishments_education_experience_skills_summary', 25), ('accomplishments_education_experience_highlights_languages_skills_summary', 20), ('accomplishments_education_experience_highlights_professional affiliations_skills_summary', 18), ('accomplishments_certifications_education_experience_highlights_skills_summary', 18), ('core qualifications_education_experience_professional 

In [15]:
content_file = "output/resume_content.txt"
with open(content_file, "w", encoding='utf-8') as f:
    for idx in range(2484):
        line = data["row"+str(idx)]
        if line:
            print(line, file=f)

In [16]:
freq_file = "output/title_freq.txt"
with open(freq_file, "w", encoding='utf-8') as f:
    for title, count in sorted(freq.items(),
                             key=lambda x: x[1], reverse=True):
        line = '{}_{}'.format(title, count)
        print(line, file=f)